In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd

In [2]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

train_df = pd.read_csv("Train.csv")
dict = {0:"healthy", 1:"fall_armyworm"}
train_df.Label = train_df.Label.map(dict)
print(train_df.head())

               Image_id          Label
0  id_02amazy34fgh2.jpg  fall_armyworm
1  id_02mh3w48pmyc9.jpg        healthy
2  id_02rpb463h9d3w.jpg        healthy
3  id_02wc3jeeao8ol.jpg  fall_armyworm
4  id_03t2hapb8wz8p.jpg  fall_armyworm


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Images",
    dataframe = train_df,
    x_col = "Image_id",
    y_col = "Label",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Images",
    dataframe = train_df,
    x_col = "Image_id",
    y_col = "Label",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 1296 validated image filenames belonging to 2 classes.
Found 323 validated image filenames belonging to 2 classes.


In [4]:
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19

base_model = VGG19(input_shape=(256,256,3),include_top=False,weights="imagenet")
base_model.trainable = True

for layer in base_model.layers:
    if layer.name == "block3_pool":
        break
    layer.trainable = False

In [5]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

def model(y):
    x = preprocess_input(y)
    x = base_model(x,training=False)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(128,activation="relu")(x)
    x = Dropout(0.2,seed=42)(x)
    x = Dense(64,activation="relu")(x)
    x = Dropout(0.2,seed=42)(x)
    output = Dense(2)(x)
    model = Model(y,output,name="transfer_vgg19")
    return model

model = model(tf.keras.Input(shape=(256,256,3)))
model.summary()
model.compile(RMSprop(learning_rate=1e-5),CategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

Model: "transfer_vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 256, 256, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 256, 256, 3)      0         
                                                                 
 vgg19 (Functional)          (None, 8, 8, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 512)            

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

if __name__=="__main__":
    checkpoint = ModelCheckpoint("fall_armyworm.h5",save_weights_only=False,save_best_only=True)
    model.fit(train_ds,epochs=5,validation_data=val_ds,callbacks=[checkpoint])
    best = load_model("fall_armyworm.h5")
    loss,acc = best.evaluate(val_ds)
    print("\nAccuracy: {:.2f} %".format(100*acc))
    print("Loss: {:.2f} %".format(100*loss))

Epoch 1/5
41/41 [==============================] - 48s 921ms/step - loss: 0.2758 - accuracy: 0.8997 - val_loss: 0.1251 - val_accuracy: 0.9659
Epoch 2/5
41/41 [==============================] - 34s 814ms/step - loss: 0.1054 - accuracy: 0.9691 - val_loss: 0.0748 - val_accuracy: 0.9628
Epoch 3/5
41/41 [==============================] - 34s 816ms/step - loss: 0.0380 - accuracy: 0.9853 - val_loss: 0.0588 - val_accuracy: 0.9845
Epoch 4/5
41/41 [==============================] - 33s 822ms/step - loss: 0.0367 - accuracy: 0.9892 - val_loss: 0.0465 - val_accuracy: 0.9845
Epoch 5/5
11/11 [==============================] - 6s 447ms/step - loss: 0.0312 - accuracy: 0.9876

Accuracy: 98.76 %
Loss: 3.12 %
